In [48]:
import pandas as pd
import ast

music_data_frame = pd.DataFrame(index = [1])
music_data_counter = 1

# получение данных о предпочтениях пользователя
user_counter = 1
while user_counter != 27000:
	try:
		users_file = open('users_data/' + str(user_counter), 'r', encoding = 'cp1251')
		readed_data = users_file.read()
		users_file.close()
		readed_data = readed_data.lower()

		# преобразуем данные в словарь
		data = ast.literal_eval(readed_data)
		user_music = data['response'][0]['music']
		user_music = user_music.split(',')
		#print(user_music)
		for artist in user_music:
			if (artist[0] == ' '): 
				artist = artist[1:]                
			if (artist not in music_data_frame.columns) and (len(str(artist)) < 20) and (len(str(artist)) > 3):
				music_data_frame[artist] = 0
				music_data_counter = music_data_frame.count(axis = 1)
				music_data_frame.loc[music_data_counter[1]] = 0
		
		for fix_artist in user_music:
			if (fix_artist[0] == ' '):
				fix_artist = fix_artist[1:]  
			for artist in user_music:
				if (artist[0] == ' '):
					artist = artist[1:]                
				
				if (fix_artist != artist) :            
					artist_index = music_data_frame.columns.get_loc(artist)
					fix_artist_index = music_data_frame.columns.get_loc(fix_artist)                   
					music_data_frame[artist][fix_artist_index + 1] += 1
					music_data_frame[fix_artist][artist_index + 1] += 1
               
		user_counter += 1
	except Exception as e:
		#print('errrroooor ' + str(e))
		user_counter += 1
		continue


columns_num = music_data_frame.count(axis = 1)

#print(music_data_frame.columns)
#print(columns_num[1])
# генерируем нужны колонки
#for i in range(2, columns_num[1]):
#	music_data_frame.loc[i] = 0

# сгенерированый csv со всеми взаимосвязями
music_data_frame.to_csv('music_matrix.csv', encoding = 'cp1251')


#df = pd.read_csv('music_matrix.csv', encoding = 'cp1251')
#cn = df.count(axis = 1)
# получаем имя колонки (queen - в данном случае)
#name = df.columns[1]

#music_data_frame['queen'].isnull() 
df = music_data_frame
#music_data_frame.columns.get_loc("чайф")
#music_data_frame
col_list = list(music_data_frame.columns.values)
#print(col_list)
for col in df:
	d = df[col].value_counts(dropna = False)
	if (len(d) > 5):
		print(d.name.upper())
		music_dict = {}
		for g in d:
			#print(col_list[g])  
			if (col_list[g] not in music_dict):
				music_dict[col_list[g]] = 1
			else:
				music_dict[col_list[g]] += 1
		print(music_dict)
    

#music_data_frame


PINK FLOYD
{'фрэнк синатра': 4, 'john surman': 1, 'pink floyd': 1, 'высоцкий': 1, 'виа под водой': 2, 'аквариум': 1, 'bjork': 1, 'mucc': 1}
BEATLES
{'muse': 1, 'смех': 1, 'beatles': 1, 'фрэнк синатра': 2, 'по настроению..': 1, 'люди лопеса': 1}
АКВАРИУМ
{'beatles': 1, 'sugarcult': 1, 'diary of dreams': 1, 'pink floyd': 2, 'виа под водой': 1}
RADIOHEAD
{'фрэнк синатра': 1, 'bregovic': 1, 'pink floyd': 1, 'ladytron': 1, 'beatles': 1, 'thunderstone': 1, 'rammstein': 1}
MUSE
{'ladytron': 1, 'piazzolla': 2, 'фрэнк синатра': 4, 'beatles': 1, 'pop-punk': 1, 'зимовье зверей': 1, '80-х': 1}
DEPECHE MODE
{'jimmy hendrix': 1, 'piazzolla': 1, 'pink floyd': 1, 'океан ельзи': 1, 'яидмг': 1, 'beatles': 1, 'фрэнк синатра': 2, 'зимовье зверей': 1}
ВЫСОЦКИЙ
{'piazzolla': 1, 'фрэнк синатра': 1, 'solas': 1, 'beatles': 1, 'башлачев': 1, 'endraum': 1}
QUEEN
{'музыка?': 1, 'avishai cohen': 1, 'piazzolla': 1, 'pink floyd': 1, 'ohmega watts': 1, 'beatles': 1, 'фрэнк синатра': 2, 'аквариум': 1, 'complex numbers